# Introduction

This IPython notebook represents my written report and source code for COMPSCI 369 Assignment 3. To generate a PDF version of this report, simply type `rake` at the command line. The generated report can be found at `report.pdf`. Please note that you will need the following software installed.

* Graphviz
* IPython 3
* \LaTeX
* pandoc
* Rake

In [ ]:
import itertools as it
import math
import random

# Question 1

![](hmm.pdf)

|       |**H**|**E**|**T**|
|:-----:|:---:|:---:|:---:|
| **B** | 0.3 | 0.15| 0.4 |
| **I** | 0.5 | 0.55| 0.1 |
| **N** | 0.2 | 0.3 | 0.5 |